In [ ]:
import glob
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import plotly.express as px
import pandas as pd
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import matplotlib as mpl
import matplotlib.animation as animation
import simplekml
import yaml
from scipy import constants
from numba import njit
from numba.typed import Dict
from tqdm.contrib.concurrent import process_map

from utils.antenna import *
from utils.sig_processor import *   
from utils.vhcl_processor import * 
from utils.config_parser import *

EXP_DATE = "2023-12-15"
EXP_DIR = glob.glob(f"../field_data/measurements/{EXP_DATE}*")

print(f"Processing date: {EXP_DATE}")

MAPBOX_API_KEY=""
## Parse this information from config file
sample_rate = 56e6
OFDM_LEN = 450
ZC_LEN = 401
OFDM_GUARD = 300

CC1 = (35.773851, -78.677010)
LW1 = (35.72747884, -78.69591754) 
LW2 = (35.72821230,-78.70093459)
H_TOWER_LW1 = 12
H_TOWER_LW2 = 10
UTC_TIME_OFF = 3
UTC_OFF_REQ = True

%matplotlib inline

In [2]:
tx_ant = Antenna() 
tx_ant.readTxAntenna()
rx_ant = Antenna() 
rx_ant.readRxAntenna()

In [3]:
def nearest(items, pivot):
    it = min(items, key=lambda x: abs(x - pivot))
    return items.index(it) 

In [4]:
class PostProcessor:
    def __init__(self, vhc_metrics, s_prc) -> None:
        self.vhc_metrics = vhc_metrics
        self.s_prc = s_prc
        
    def process_meas(self, path): 
        r = np.load(path, allow_pickle=True) 
        ind_n = nearest([v.time for v in self.vhc_metrics], datetime.fromtimestamp(r["time_info"].item()) + timedelta(hours=UTC_TIME_OFF))
        res = self.s_prc.process(r["rx_time"], r["rcv"][0], self.vhc_metrics[ind_n])
        return res


In [5]:
def find_closest_log(log_directory, input_date_str):
    # Parse the input date string
    input_date_format = "%Y-%m-%d_%H_%M"
    input_date_str = input_date_str.split('/')[-1]  # Extract the date part from the path
    input_date = datetime.strptime(input_date_str, input_date_format)
    input_date_format = "%Y-%m-%d_%H_%M_%S"

    closest_log = None
    smallest_diff = None
    
    # Iterate through the files in the log directory
    for file_name in os.listdir(log_directory):
        if file_name.endswith("_vehicleOut.txt"):
            # Extract the date and time from the file name
            file_date_str = file_name.split('_vehicleOut')[0]
            file_date = datetime.strptime(file_date_str, input_date_format)
            
            # Calculate the time difference
            time_diff = abs((file_date - input_date).total_seconds())
            
            if smallest_diff is None or time_diff < smallest_diff:
                closest_log = file_name
                smallest_diff = time_diff
                
    return log_directory + closest_log

In [6]:
# @njit(parallel=True) 
def process_date(path, cfg, ext_loc=None):
    measurements = sorted(glob.glob(path+"/*.npz"), key=os.path.getmtime)
    vhc_log = find_closest_log("../field_data/vehicle_logs/",path)
    locs_dir = glob.glob(vhc_log) 

    if not locs_dir:
        return None
    
    print(path, locs_dir)

    vehicle = Vehicle_Processor(cfg)
    vehicle.read_vehicle_data(locs_dir[0])

    v_metrics = vehicle.get_metrics()
    
    r = np.load(measurements[0], allow_pickle=True) 

    s_prc = SigProcessor(cfg, r["ref"][:ZC_LEN], None, ZC_LEN*4)
    p_prc = PostProcessor(v_metrics, s_prc)
    
    # for m in tqdm(measurements):
    #     process_meas(m, vehicle, cfg, s_prc) 
 
    r = process_map(p_prc.process_meas, measurements, max_workers=os.cpu_count())
    # Parallel(n_jobs=20, require='sharedmem')(delayed(process_meas)(m, loc, d, cfg, tx_ant, rx_ant) for m in tqdm(measurements))
    return r 
    

In [7]:
def parseConfig(path):
    with open(path, "r") as stream:
        config = yaml.safe_load(stream)
    return config

In [8]:
EXP_DIR_FLT = []
for m in EXP_DIR:
    fls = glob.glob(m+"/*.npz")
    if len(fls) > 500:
        EXP_DIR_FLT.append(m)
EXP_DIR_FLT.sort()

In [ ]:
EXP_DIR_FLT

In [10]:
def conv_arr_to_df(arr):
    df = pd.DataFrame()
    for i in range(len(arr)):
        if arr[i]:
            if arr[i].detected:
                d_it = arr[i].__to_dict__()
                d_it["vehicle"] = pd.DataFrame.from_dict(arr[i].vehicle.__to_dict__(), orient="index")
                df = pd.concat([df, pd.DataFrame.from_dict(d_it, orient="index")], axis=1)

    return df.T

In [11]:
def df_type_corr(df):
    df["lat"] = df["lat"].astype(np.float32)
    df["lon"] = df["lon"].astype(np.float32)
    df["alt"] = df["alt"].astype(np.float32)
    df["dist"] = df["dist"].astype(np.float32)
    df["h_dist"] = df["h_dist"].astype(np.float32)
    df["v_dist"] = df["v_dist"].astype(np.float32)   

    df["avgPower"] = df["avgPower"].astype(np.float32)
    return df

In [12]:
def pick_key_from_metrics(metrics, key):
    r = []
    for i in range(len(metrics)):
        if metrics[i]:
            if metrics[i].detected:
                r.append(getattr(metrics[i], key))
    return r

In [13]:
PROCESS = True

prcsd = {
    "resultDir": [],
    "meas": [],
    "config": [],
    "freq": [],
    "waveType": []
}

In [ ]:
m = EXP_DIR_FLT[6]
RESULT_DIR = f"../field_data/post-results/{m.split('/')[-1]}/"

if not os.path.exists(RESULT_DIR):
    os.makedirs(RESULT_DIR)  

## Check if pickle file exists 

cfg = Config(m + "/config.yaml")
# if not os.path.exists(RESULT_DIR + "processed.pkl") and PROCESS:
if True:
    res = process_date(m, cfg) 
    df = conv_arr_to_df(res)
    df = df.convert_dtypes()
    df = df_type_corr(df)
    df.reset_index(inplace=True)
    df.to_pickle(RESULT_DIR + "processed.pkl")
    prcsd["resultDir"].append(RESULT_DIR)
    prcsd["meas"].append(res) 
# cfg = parseConfig(m + "/config.yaml")
else:
    df = pd.read_pickle(RESULT_DIR + "processed.pkl")

In [ ]:
# Run only if you want to process all the measurements
for m in tqdm(EXP_DIR_FLT):
    RESULT_DIR = f"../field_data/post-results/{m.split('/')[-1]}/"

    if not os.path.exists(RESULT_DIR):
        os.makedirs(RESULT_DIR)  
    cfg = Config(m + "/config.yaml")
    if not os.path.exists(RESULT_DIR + "processed.pkl") and PROCESS:
    # if True:
        res  = process_date(m, cfg) 
        df = conv_arr_to_df(res)
        df = df.convert_dtypes()
        df = df_type_corr(df)
        df.reset_index(inplace=True)
        df.to_pickle(RESULT_DIR + "processed.pkl")

        freq = cfg.USRP_CONF.CENTER_FREQ / 1e6
        waveType = cfg.WAVEFORM
        
        prcsd["freq"].append(freq)
        prcsd["waveType"].append(waveType)
        prcsd["resultDir"].append(RESULT_DIR)
        prcsd["config"].append(cfg)
        prcsd["meas"].append(res)
    else:
        df = pd.read_pickle(RESULT_DIR + "processed.pkl")

In [22]:
freq = cfg.USRP_CONF.CENTER_FREQ / 1e6
waveType = cfg.WAVEFORM

In [ ]:
plt.plot(df["time"], df["avgPower"])

In [ ]:
df["alt"]

In [ ]:
plt.plot(df["time"], df["lat"])

In [ ]:
# Read all the processed data

for m in tqdm(EXP_DIR_FLT):
    RESULT_DIR = f"../field_data/post-results/{m.split('/')[-1]}/"
    
    cfg = Config(m + "/config.yaml")
    df = pd.read_pickle(RESULT_DIR + "processed.pkl")
    
    freq = cfg.USRP_CONF.CENTER_FREQ / 1e6
    waveType = cfg.WAVEFORM
    
    prcsd["freq"].append(freq)
    prcsd["waveType"].append(waveType)
    prcsd["resultDir"].append(RESULT_DIR)
    prcsd["config"].append(cfg)
    prcsd["meas"].append(df)

In [41]:
from plotly.express.colors import sample_colorscale

x = np.linspace(0, 1, 50)

def list_rgba_colors(alpha=0.1):
    # colors = px.colors.get_colorscale("inferno")
    colors = sample_colorscale('jet', list(x))
    rgba_colors = [color.replace("rgb","rgba").replace(")", f", {alpha})") for color in colors]
    return rgba_colors

opac_colors = list_rgba_colors(0.15) 
fl_colors = list_rgba_colors(1.0)

In [ ]:
plt.plot(df["time"], df["freq_offset"])

# Plots

In [ ]:

from tkinter import font

fig = go.Figure()

alpha = 0.2

for i in range(len(prcsd["meas"])):
    if prcsd["meas"][i]["alt"].max() < 40:  # 30 Meter plan
        if len(prcsd["meas"][i]) < 100:
            continue
        prcsd["meas"][i] = prcsd["meas"][i].sort_values(by="time")
        fig.add_trace(go.Scatter(x=prcsd["meas"][i]["dist"], y=prcsd["meas"][i]["avgPower"], mode='lines+markers', marker={
                      "color": opac_colors[i]}, name=f"{prcsd['freq'][i]} MHz {prcsd['waveType'][i]}"))
        avg_power_ewma = prcsd["meas"][i]["avgPower"].ewm(alpha=0.2).mean()
        fig.add_trace(go.Scatter(x=prcsd["meas"][i]["dist"], y=avg_power_ewma, mode='lines', marker={
                      "color": fl_colors[i]}, name=f"{prcsd['freq'][i]} MHz {prcsd['waveType'][i]} EWMA alpha={alpha} "))


fig.update_layout(width=1920, height=1080, title="Average Power vs Distance", xaxis_title="Distance (m)", yaxis_title="Average Power (dBm)", legend=dict(
    yanchor="bottom", y=0.01, xanchor="left", x=0.01),
    font=dict(family="Courier New, monospace", size=32),
    legend_title_text="Frequency (MHz) Waveform")

fig.show()
pio.write_image(fig, RESULT_DIR + "/_avg_power_dist.png",
                width=1920, height=1080)

In [ ]:
# Avg power at different altitudes 
fig = go.Figure()
altitudes = np.array([30, 60, 90])
for i in range(len(prcsd["meas"])):
    if len(prcsd["meas"][i]) < 100:
        continue
    prcsd["meas"][i] = prcsd["meas"][i].sort_values(by="time")
    alt = np.floor(prcsd["meas"][i]["alt"].max())
    fig.add_trace(go.Scatter(x=prcsd["meas"][i]["dist"], y=prcsd["meas"][i]["avgPower"], mode='lines+markers', marker={
                    "color": opac_colors[i*4]}, name=f"{prcsd['freq'][i]} MHz Altitude = {alt} m"))
    
    avg_power_ewma = prcsd["meas"][i]["avgPower"].ewm(alpha=0.2).mean()
    fig.add_trace(go.Scatter(x=prcsd["meas"][i]["dist"], y=avg_power_ewma, mode='lines', marker={
                    "color": fl_colors[i*4]}, name=f"{prcsd['freq'][i]} MHz EWMA <br>Alpha={alpha} Altitude = {alt} m")) 
    
fig.update_layout(width=1920, height=1080,  xaxis_title="Distance (m)", yaxis_title="Average Power (dBm)", legend=dict(
    yanchor="top", y=0.92, xanchor="left", x=1.01),
    font=dict(family="Courier New, monospace", size=20))

fig.show()

pio.write_image(fig, RESULT_DIR + "/_avg_power_dist_alt.png")

In [ ]:
os.system(f"dolphin  {RESULT_DIR} &")

In [23]:
df = prcsd["meas"][0]
freq = prcsd["freq"][0]
waveType = prcsd["waveType"][0]

In [ ]:
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=df.lat,
            y=df.lon,
            z=df.alt,
            mode="markers",
            marker=dict(
                size=12,
                color=df.avgPower,  # set color to an array/list of desired values
                colorscale="Thermal",  # choose a colorscale
                opacity=0.8,
            ),
        )
    ]
)

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
# 3D Plot of Power

RESULT_DIR = prcsd["resultDir"][0]
inferno = mpl.colormaps['jet'].resampled(320)
kml = simplekml.Kml()
# maxPw = df["power"].argmax() 

## TODO interpolate **
prev_loc = None 
for row in df.iterrows():
    ls = kml.newlinestring()
    ls.altitudemode = simplekml.AltitudeMode.relativetoground
    ls.style.linestyle.width = 10
    rgb_r = inferno((row[1]["avgPower"]- df["avgPower"].min()) / (df["avgPower"].max() - df["avgPower"].min())) 
    ls.style.linestyle.color = simplekml.Color.rgb(int(rgb_r[0]*255), int(rgb_r[1]*255), int(rgb_r[2]*255))
    if not prev_loc:
        ls.coords = [(row[1]["lon"], row[1]["lat"], row[1]["alt"])] 
    else: 
        ls.coords = [(row[1]["lon"], row[1]["lat"], row[1]["alt"]), prev_loc] 
    prev_loc = (row[1]["lon"], row[1]["lat"], row[1]["alt"])

kml.save(RESULT_DIR + "Power_GPS3D.kml")

In [ ]:
inferno = mpl.colormaps['jet'].resampled(320)

In [ ]:
# 3D Plot of Path Loss
RESULT_DIR = prcsd["resultDir"][0]
viridis = mpl.colormaps['viridis_r'].resampled(24)
kml = simplekml.Kml()
maxPL = df["avg_pl"].argmax() 
prev_loc = None
for row in df.iterrows():
    ls = kml.newlinestring()
    ls.altitudemode = simplekml.AltitudeMode.relativetoground
    ls.style.linestyle.width = 10
    rgb_r = viridis(row[1]["avg_pl"]/maxPL) 
    ls.style.linestyle.color = simplekml.Color.rgb(int(rgb_r[0]*255), int(rgb_r[1]*255), int(rgb_r[2]*255))
    if not prev_loc:
        ls.coords = [(row[1]["lon"], row[1]["lat"], row[1]["alt"])] 
    else: 
        ls.coords = [(row[1]["lon"], row[1]["lat"], row[1]["alt"]), prev_loc] 
    prev_loc = (row[1]["lon"], row[1]["lat"], row[1]["alt"])


kml.save(RESULT_DIR + "PathLoss_GPS3D.kml")

In [ ]:
px.set_mapbox_access_token(
    MAPBOX_API_KEY
)
RESULT_DIR = prcsd["resultDir"][p_i]

fig = px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    color="avg_pl",
    labels={"avg_pl": "Path Loss(dB)"},
    # title="Power Distribution Between CC1 and PN",
    mapbox_style="satellite",
    color_continuous_scale=px.colors.sequential.thermal,
    size_max=11,
    size="lat", # Just for making the markers bigger
    zoom=16.2,
    width=1400,
    height=1000
)

fig.update_coloraxes(showscale=True)

fig.show()
pio.write_image(fig, RESULT_DIR + "PathLoss_GPS.png", scale=1, width=1400, height=1000)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df.time, y=df.dist, mode="markers", name="GPS Based Distance")
)

fig.add_trace(
    go.Scatter(
        x=df.time,
        y=df.est_dist,
        mode="markers",
        name="Estimated Distance from the Received Signal",
    )
)

fig.update_layout(
    autosize=False,
    width=2400,
    height=1000,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    # title_text='GPS Distance vs Estimated Distance between CC1 and PN',
    title_x=0.5,
    font=dict(family="Courier New, monospace", size=36),
    xaxis_title="Time (s)",
    yaxis_title="Distance (m)",
)

fig.update_traces(marker=dict(size=16), selector=dict(mode="markers"))
fig.update_layout(
    legend=dict(
        yanchor="top",
        y=1.15,
        xanchor="left",
        font=dict(family="Courier New, monospace", size=36, color="black"),
        x=0.01,
        bgcolor="silver",
    ),
    plot_bgcolor="white",
)

fig.update_xaxes(showline=True, linewidth=2, linecolor="black")
fig.update_yaxes(showline=True, linewidth=3, linecolor="black")

fig.show()
pio.write_image(
    fig, RESULT_DIR + "EstimatedDist_vs_GPS.png", scale=6, width=1800, height=1000
)


In [ ]:
fig = px.scatter(df, x="time", y="dist", color="avgPower", labels={"avgPower": "Power (dB)", "dist":"Distance (m)", "time": "Time (s)"}, width=2500, height=1000, title="Distance vs time with Power")
pio.write_image(fig, RESULT_DIR+"Power_vs_Distance_time.png",scale=6, width=2500, height=1000)
fig.show()

In [ ]:
RESULT_DIR = prcsd["resultDir"][0]
fig = px.scatter(df, x="dist", y="avgPower", labels={"avgPower": "Power (dB)", "dist":"Distance (m)", "time": "Time (s)"}, width=2500, height=1000, trendline="lowess", trendline_options=dict(frac=0.1), title="Distance vs Power")
fig.show()
pio.write_image(fig, RESULT_DIR+"Power_vs_Distance.png",scale=6, width=2500, height=1000)

In [ ]:
RESULT_DIR = prcsd["resultDir"][p_i]
fig = px.scatter(df, x="dist", y="rflct", labels={"rfclt": "Reflection Path Distance Difference", "dist":"Distance (m)", "time": "Time (s)"}, width=2500, height=1000, trendline="lowess", trendline_options=dict(frac=0.1), title="Numvwer of Reflections vs Distance")
fig.show()
pio.write_image(fig, RESULT_DIR+"Reflection_vs_Distance.png",scale=6, width=2500, height=1000)

In [ ]:
rflcts = []
for i in range(len(prcsd["meas"][6])):
    rflcts.extend(prcsd["meas"][6]["rflcts"][i])

rflcts = [x for x in rflcts if x < 1000]

hist, bin_edges = np.histogram(rflcts, bins=10, density=True)
# hist = hist * np.diff(bin_edges)
cdf = np.cumsum(hist * np.diff(bin_edges))

In [ ]:
# CDF of reflections 
fig = px.ecdf(x=rflcts, width=1200, height=500, title="CDF of Reflection Path Distance Difference", markers=True, lines=False, marginal="histogram")
fig.update_layout(
    autosize=True,
    width=1200,
    height=700,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    title_text='CDF of Reflection Path Distance Difference',
    title_x=0.5,
    font=dict(family="Courier New, monospace", size=24),
    xaxis_title="Reflection Path Distance Difference (meters)",
    yaxis_title="Probability",
)
fig.show()


In [ ]:
n_of_reflcts = []
for i in range(len(prcsd["meas"][0])):
    n_of_reflcts.append(len(prcsd["meas"][0]["rflcts"][i]))

n_of_reflcts = [x for x in n_of_reflcts if x < 5]

fig = px.ecdf(x=n_of_reflcts, width=1200, height=700, title="CDF of Number of Reflections", markers=True, lines=False, marginal="histogram")

fig.update_layout(
    autosize=False,
    width=1200,
    height=500,
    margin=dict(l=50, r=50, b=100, t=100, pad=1),
    title_text='CDF of Number of Reflections',
    title_x=0.25,
    font=dict(family="Courier New, monospace", size=24),
    xaxis_title="Number of Reflections",
    yaxis_title="Probability",
)
fig.show()
## TODO number of reflections vs distance

In [ ]:
freq = prcsd["meas"][0]["freq"][1]
waveType = prcsd["meas"][0]["waveType"][1]
df = prcsd["meas"][0]
RESULT_DIR = prcsd["resultDir"][0]
df["l_rflcts"] = df[df["l_rflcts"] < 5]["l_rflcts"]
fig = px.scatter(x=df["time"], y=df["l_rflcts"], labels={"y": "Reflection Path Distance Difference", "x":"Time (s)", "time": "Time (s)"}, width=2500, height=1000, trendline="lowess", trendline_options=dict(frac=0.1), title="Distance vs Power")
fig.update_layout(
    autosize=False,
    width=2500,
    height=1000,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    title_text='Number of Reflections vs Time',
    title_x=0.5,
    font=dict(family="Courier New, monospace", size=36),
    xaxis_title="Time (s)",
    yaxis_title="Number of Reflections",
)
fig.show()
pio.write_image(fig, RESULT_DIR+"Reflection_vs_Distance.png",scale=6, width=2500, height=1000)

In [ ]:
RESULT_DIR = prcsd["resultDir"][p_i]
df_f = df[(df["dist"] > 100)].copy()
df_f = df_f[(df_f["freq_offset"].abs() < 4e3)].copy()
fig = px.scatter(df, x="time", y="freq_offset", labels={"freq_offset": "Frequency Offset (Hz)", "dist":"Distance (m)", "time": "Time (s)"}, width=1500, height=750, trendline="lowess", trendline_options=dict(frac=0.1), title=f"Freq. Offset vs Time | Average Frequency Offset: {df["freq_offset"].abs().mean()} Hz")
fig.show()
pio.write_image(fig, RESULT_DIR+"Freq_off_vs_time.png",scale=6, width=1200, height=800)

In [ ]:
df2 = df.copy()

In [ ]:
fig = px.scatter(df2, x="dist", y="avg_pl", color="avgPower", labels={"avg_pl": "Path Loss (dB)", "dist":"Distance (m)", "time": "Time (s)"}, width=2000, height=1000, title="Distance vs Path Loss")
fig.show()
# pio.write_image(fig, RESULT_DIR+"PathLoss_vs_Distance.png",scale=6, width=2500, height=1000)
fig.write_image(RESULT_DIR+"PathLoss_vs_Distance.png", width=2000, height=1000)

In [ ]:
# TODO Make whole process one function to update df

# Path Loss

In [20]:
current_meas = 7 
df = prcsd["meas"][current_meas-1]
cfg = prcsd["config"][current_meas-1]
freq = cfg.USRP_CONF.CENTER_FREQ / 1e6
waveType = cfg.WAVEFORM
RESULT_DIR = prcsd["resultDir"][current_meas-1]

In [21]:
def RMa_3GPP(d_2d, d_3D, fc, h_bs = 10, h_ue = 1.5, h = 5, w = 10):
        PL_RMa_LOS, PL_RMa_NLOS = 0.0, 0.0
        
        if h_ue < 1 : 
            return np.nan
        d_BP = 2*np.pi*h_bs*h_ue*fc 
        if d_BP == 0.0 or d_3D == 0.0:
            return np.nan
        
        PL_1 = 20*np.log10(40*np.pi*d_3D*fc/3) +  min(0.03*h**1.72 , 10)*np.log10(d_3D) - min(0.044*h**1.72 , 14.77) + 0.002*np.log10(h) *d_3D
        PL_2 = PL_1 + 40*np.log10(d_3D/d_BP)
        if  10 < d_2d and d_2d <= d_BP:
            PL_RMa_LOS = PL_1 + 4 * np.random.normal(0, 1) # Shadow fading
        elif d_2d < 10:
            PL_RMa_Los = np.nan
        elif d_BP < d_2d and d_2d <= 10*1000:
            PL_RMa_LOS = PL_2 + 6 * np.random.normal(0, 1)
            
        PL_RMa_NLOS_2 = 161.04 - 7.1*np.log10(40*w) + 7.5*np.log10(h) -(24.37 - 3.7*(h/h_bs)*(h/h_bs))*np.log10(h_bs) + (43.42 - 3.1 * np.log10(h_bs))*(np.log10(d_3D)-3) + 20*np.log10(fc) - (3.2*np.log10(11.75*h_ue)**2 - 4.97)
        
        if 10 < d_2d and d_2d <= 5*1000:
            PL_RMa_NLOS = max(PL_RMa_LOS, PL_RMa_NLOS_2) + 8
            
        
        return PL_RMa_LOS

In [22]:
def calc_pathloss(df, cfg):

    freq = cfg.USRP_CONF.CENTER_FREQ / 1e6

    df = df.sort_values("time")
    dist = df["dist"].values

    tx_ant_gain = tx_ant.getGain(df.aod_theta-1.7, df.aod_phi)
    rx_ant_gain = rx_ant.getGain(df.aoa_theta+1.9+df.yaw, df.aoa_phi-.07+df.pitch)

    lmbd = 3e8 / (freq*1e6) 

    pl_free = 10*np.log10((4*np.pi*dist)/(lmbd)) - (tx_ant_gain + rx_ant_gain)

    d_los = dist 
    d_ref = np.sqrt((df.alt.values + H_TOWER_LW1) ** 2 + df.h_dist.values ** 2) 
    phi = 2* np.pi * (d_ref - d_los) / lmbd 
    G_LOS = 1 
    G_REF = 0.3 
    REF_COEFF = 0.4
    los_coef = np.sqrt(G_LOS) / d_los 
    ref_coef = np.sqrt(G_REF) / d_ref * np.exp(-1j*phi) * REF_COEFF
    pl_2ray = -20*np.log10(lmbd * np.abs(los_coef + ref_coef)/(4*np.pi)) - (tx_ant_gain + rx_ant_gain)
    
    new_model = pl_2ray + np.random.lognormal(0, 1e-10, len(pl_2ray))

    ## Moving average trendline for avg_pl 
    df["avg_pl_rolling"] = df["avg_pl"].rolling(window=10).mean()

    ## EWMA moving average trendline for avg_pl 
    alpha = 0.2
    df["avg_pl_ewma"] = df["avg_pl"].ewm(alpha=alpha, adjust=True).mean()
    
    pl_RMA_LoS  = [RMa_3GPP(row.h_dist,row.dist, freq/1e3,h_ue=row.alt) for i, row in df.iterrows()] - (tx_ant_gain + rx_ant_gain) 
    
    return pl_RMA_LoS, pl_2ray, pl_free, new_model

In [ ]:
data = { 
    "Center Frequency (MHz)": [],
    "Altitude (m)": [],
    "Measured Mean (dB)": [],
    "Measured STD (dB)": [],
    "3GPP RMa Mean (dB)": [],
    "3GPP RMa STD (dB)": [],
    "2-Ray Mean (dB)": [],
    "2-Ray STD (dB)": [],
    "Free Space Mean (dB)": [],
    "Free Space STD (dB)": [],
    "New Model Mean (dB)": [],
    "New Model STD (dB)": []
}

for i in range(len(prcsd["meas"])):
    rma_loss, pl_2ray, pl_free,new_model = calc_pathloss(prcsd["meas"][i], prcsd["config"][i])
    freq = prcsd["config"][i].USRP_CONF.CENTER_FREQ / 1e6
    print(f"Center Freq: {freq} MHz | Altitude: {prcsd['meas'][i]['alt'].max()} m | Path Loss Mean: {prcsd['meas'][i]['avg_pl'].mean()} dB | Path Loss STD: {prcsd['meas'][i]['avg_pl'].std()} dB ")
    print(f"3GPP RMA Path Loss Mean: {np.nanmean(rma_loss)} dB | 3GPP RMA Path Loss STD: {np.nanstd(rma_loss)} dB ")
    print(f"2-Ray Path Loss Mean: {np.nanmean(pl_2ray)} dB | 2-Ray Path Loss STD: {np.nanstd(pl_2ray)} dB ")
    print(f"Free Space Path Loss Mean: {np.nanmean(pl_free)} dB | Free Space Path Loss STD: {np.nanstd(pl_free)} dB ")
    
    data["Center Frequency (MHz)"].append(freq)
    data["Altitude (m)"].append(prcsd["meas"][i]["alt"].max())
    data["Measured Mean (dB)"].append(prcsd["meas"][i]["avg_pl"].mean())
    data["Measured STD (dB)"].append(prcsd["meas"][i]["avg_pl"].std())
    data["3GPP RMa Mean (dB)"].append(np.nanmean(rma_loss))
    data["3GPP RMa STD (dB)"].append(np.nanstd(rma_loss))
    data["2-Ray Mean (dB)"].append(np.nanmean(pl_2ray))
    data["2-Ray STD (dB)"].append(np.nanstd(pl_2ray))
    data["Free Space Mean (dB)"].append(np.nanmean(pl_free))
    data["Free Space STD (dB)"].append(np.nanstd(pl_free))
    data["New Model Mean (dB)"].append(np.nanmean(new_model))
    data["New Model STD (dB)"].append(np.nanstd(new_model))
    

In [ ]:
df_pl = pd.DataFrame(data)
df_pl

In [ ]:
df_pl = pd.DataFrame(data)
df_pl.sort_values("Altitude (m)", inplace=True)
# Line plot for Path Loss Mean vs. Altitude for each model
df_pl = df_pl.drop(3)
fig = go.Figure() 

colors = ['blue', 'red', 'green', 'purple']  # Base colors for models
line_styles = ['solid', 'dot', 'dash']

models = ["Measured", "3GPP RMa", "2-Ray", "Free Space"]
for model, color in zip(models, colors):
    for i, freq in enumerate(sorted(df_pl["Center Frequency (MHz)"].unique())):
        trace_df = df_pl[df_pl["Center Frequency (MHz)"] == freq]
        fig.add_trace(
            go.Scatter(
                x=trace_df["Altitude (m)"],
                y=trace_df[f"{model} STD (dB)"],
                mode='lines+markers',
                name=f"{model} at {freq} MHz",
                line=dict(color=color, dash=line_styles[i]),
                legendgroup=model
            )
        )

fig.update_layout(
    autosize=False,
    width=1920,
    height=1500,
    title_x=0.5,
    font=dict(family="Courier New, monospace", size=30),
    xaxis_title="Altitude (m)",
    yaxis_title="Path Loss σ<sup>2</sup>",
    legend=dict(
        x=0.01,  # Adjust legend position
        y=1.15,  # Above the plot
        orientation='h',  # Horizontal legend
        font=dict(size=30),  # Smaller font size for legend items
        traceorder='grouped'  # Group legend entries
    ),
    plot_bgcolor='rgba(230, 236, 245, 0.5)'
)

fig.write_image(RESULT_DIR + "PathLoss_STD_vs_Altitude.png", width=1920, height=1080)
fig.show()

# open dolphin 
os.system(f"dolphin  {RESULT_DIR} &")

In [26]:
# def calc free space 
df = df.sort_values("time")
dist = df["dist"].values

tx_ant_gain = tx_ant.getGain(df.aod_theta-1.7, df.aod_phi)
rx_ant_gain = rx_ant.getGain(df.aoa_theta+1.9+df.yaw, df.aoa_phi-.07+df.pitch)

lmbd = 3e8 / (freq*1e6) 

pl_free = 10*np.log10((4*np.pi*dist)/(lmbd)) - (tx_ant_gain + rx_ant_gain)

d_los = dist 
d_ref = np.sqrt((df.alt.values + H_TOWER_LW1) ** 2 + df.h_dist.values ** 2) 
phi = 2* np.pi * (d_ref - d_los) / lmbd 
G_LOS = 1 
G_REF = 0.3 
REF_COEFF = 0.4
los_coef = np.sqrt(G_LOS) / d_los 
ref_coef = np.sqrt(G_REF) / d_ref * np.exp(-1j*phi) * REF_COEFF
pl_2ray = -20*np.log10(lmbd * np.abs(los_coef + ref_coef)/(4*np.pi)) - (tx_ant_gain + rx_ant_gain)


## Moving average trendline for avg_pl 
df["avg_pl_rolling"] = df["avg_pl"].rolling(window=10).mean()

## EWMA moving average trendline for avg_pl 
alpha = 0.2
df["avg_pl_ewma"] = df["avg_pl"].ewm(alpha=alpha, adjust=True).mean()

In [27]:
## TODO add g_ref and ref_coeff into parameters to optimize 

def mse(initial_params):
    aod_theta_off, aoa_theta_off, initial_power_off = initial_params
    tx_ant_gain = tx_ant.getGain(df.aod_theta+aod_theta_off, df.aod_phi)
    rx_ant_gain = rx_ant.getGain(df.aoa_theta+df.yaw+aoa_theta_off, df.aoa_phi+df.pitch)
    
    pl_2ray = -20*np.log10(lmbd * np.abs(los_coef + ref_coef)/(4*np.pi)) - (tx_ant_gain + rx_ant_gain)
    
    return np.mean((df["avg_pl"] - (pl_2ray + initial_power_off))**2)


def rmse(initial_params):
    return np.sqrt(mse(initial_params))

In [ ]:
fig = go.Figure()

rmse_vls = []
degs = np.arange(-np.pi, np.pi, 0.1)
for deg in degs:
    best_rmse = np.inf
    for deg2 in degs:
        tx_ant_gain = tx_ant.getGain(df.aod_theta+deg, df.aod_phi)
        rx_ant_gain = rx_ant.getGain(df.aoa_theta+df.yaw+deg2, df.aoa_phi-.07+df.pitch)
        
        pl_free = 10*np.log10((4*np.pi*dist)/(lmbd)) - (tx_ant_gain + rx_ant_gain)
        pl_2ray = -20*np.log10(lmbd * np.abs(los_coef + ref_coef)/(4*np.pi)) - (tx_ant_gain + rx_ant_gain)
        rmse = np.sqrt(np.mean((df["avg_pl"] - optimal_offset[2] - pl_2ray)**2)) 

        if rmse < best_rmse:
            best_rmse = rmse
            best_pl_2ray = pl_2ray
    rmse_vls.append(best_rmse)
    
fig.add_trace(go.Scatter(x=degs, y=rmse_vls, mode='lines+markers', name="RMSE vs Degrees")) 
fig.update_layout(title="RMSE vs Antenna Azimuth Offset", xaxis_title="Offset (radians)", yaxis_title="RMSE", )

fig.show()

In [29]:
pio.write_image(fig, RESULT_DIR + "RMSE_vs_Azimuth.png", scale=6, width=1920, height=1080)

In [ ]:
initial_params = [-3,3, 15]
bounds = [(-3,3), (-3,3), (-30, 30)]
step_size = [0.01, 0.01, 0.01]

aod_theta_sp = np.linspace(-3,3, 20) 
aoa_theta_sp = np.linspace(-3,3, 20)
offs = np.linspace(-30,30, 20)

min_mse = np.inf 
params = []

for x in aod_theta_sp:
    for y in aod_theta_sp:
        for z in offs:
            err = rmse([x, y, z]) 
            if err < min_mse:
                min_mse = err 
                params = [x,y,z]
            

optimal_offset = params
print(f"Optimal Offset: {optimal_offset}")

In [ ]:
from scipy.optimize import minimize
initial_params = [-3,3, 15]
bounds = [(-3,3), (-3,3), (-30, 30)]

res = minimize(rmse, initial_params, bounds=bounds, method="trust-constr")

print(res.x)

In [67]:
optimal_offset = res.x

In [ ]:
# params = [-1.7, 1.9, 18.5] 
params =  [-3, 2.05, -23.5263]
# params = [-3, 2.68, -20.526] 
print(rmse(params))
print(rmse(optimal_offset))
optimal_offset = params

In [35]:
tx_ant_gain = tx_ant.getGain(df.aod_theta+optimal_offset[0], df.aod_phi)
rx_ant_gain = rx_ant.getGain(df.aoa_theta+optimal_offset[1]+df.yaw, df.aoa_phi-.07+df.pitch)
pl_2ray = -20*np.log10(lmbd * np.abs(los_coef + ref_coef)/(4*np.pi)) - (tx_ant_gain + rx_ant_gain) 
pl_free = 20*np.log10((4*np.pi*dist)/(lmbd)) - (tx_ant_gain + rx_ant_gain) 

# Variance should change over window 
window_size = 10
std_dev_shadowing = 0.8

pl_free_log_normal = pl_free + std_dev_shadowing * np.random.normal(0, std_dev_shadowing, len(pl_free))
pl_2ray_log_normal = pl_2ray + std_dev_shadowing * np.random.normal(0, std_dev_shadowing, len(pl_2ray))

In [36]:
def rma_pl_3GPP(f_mhz, d, h_bs, h_ue, h=5, w=10):
    """
    Calculate the RMa path loss using the 3GPP model for rural macro cells.
    
    Parameters:
    f (float): frequency in MHz
    d (float): distance between base station and user equipment in km
    h_bs (float): height of the base station in meters
    h_ue (float): height of the user equipment in meters
    h: avg. height of the building in meters
    w: avg. width of the street in meters
    
    Returns:
    float: path loss in dB
    """

    # Frequency factor
    c = constants.speed_of_light
    
    # Antenna heights
    h_bs_eff = max(35, h_bs)  # Effective base station height
    h_ue_eff = max(1.5, h_ue)  # Effective user equipment height
    
    f = f_mhz * 1e3
    
    # Distance factor
    d_bp = 2 * np.pi * h_bs_eff * h_ue_eff * f_mhz / c / 1e3  # Breakpoint distance in km
    
    # Path loss calculation
    if 10 < d < d_bp:
        # Free space path loss
        pl1 = 20 * np.log10(40*np.pi*d*f/3) + np.min(0.03*h, 10) * np.log10(d) - np.min(0.044*h, 14.77) + 0.002*np.log10(h)*d  
    elif d < 10:
        pl = np.nan
    else:
        # Two-ray ground reflection model (assuming the general rural environment)
        pl = 40 * np.log10(d) + 13.47 + 20 * np.log10(h_bs_eff * h_ue_eff) - 3.2 * (np.log10(11.75 * h_ue_eff))**2 - 4.97

    return pl


In [ ]:
(df["avg_pl_ewma"] - optimal_offset[2]).var()

In [38]:
pl_RMA_LoS  = [RMa_3GPP(row.h_dist,row.dist, freq/1e3,h_ue=row.alt) for i, row in df.iterrows()] - (tx_ant_gain + rx_ant_gain) 
# pl_RMA_LoS_2 = [rma_pl_3GPP(freq, row.dist, H_TOWER_LW1, row.alt) for i, row in df.iterrows()] - (tx_ant_gain + rx_ant_gain) 


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["time"], y=df["avg_pl"]-optimal_offset[2], mode="markers", name="Path Loss (dB)", marker_color=opac_colors[0]))
fig.add_trace(go.Scatter(x=df["time"], y=df["avg_pl_ewma"]-optimal_offset[2], name=f"Path Loss (dB) Moving Average α={alpha}", marker_color=fl_colors[0]))
# fig = go.Figure()
fig.add_trace(go.Scatter(x=df["time"]+1, 
              y=pl_free, name="Free Space Path Loss", marker_color=opac_colors[12]))
# fig.add_trace(go.Scatter(x=df["time"], y=pl_free+(tx_ant_gain+rx_ant_gain), name="Free Space Path Loss wo Antenna Gain"))
fig.add_trace(go.Scatter(x=df["time"]+1, y=pl_2ray, name="2-Ray Path Loss", marker_color=opac_colors[36]))

fig.add_trace(go.Scatter(x=df["time"], y=pl_free_log_normal, name="Free Space Path Loss (Log Normal)", marker_color=fl_colors[12]))
fig.add_trace(go.Scatter(x=df["time"], y=pl_2ray_log_normal, name="2-Ray Path Loss (Log Normal)", marker_color=fl_colors[36]))

fig.add_trace(go.Scatter(x=df["time"], y=pl_RMA_LoS, name="3GPP RMa LoS Path Loss w/shadowing", marker_color=fl_colors[46]))


fig.update_layout(
    autosize=False,
    width=1920,
    height=1080,
    title_text='Path Loss vs Time',
    title_x=0.5,
    font=dict(family="Courier New, monospace", size=12),
    xaxis_title="Time (s)",
    yaxis_title="Path Loss (dB)",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.add_trace(go.Scatter(
    x=df.time,
    y=df.alt,
    name="Altitude",
    yaxis="y2"  # Assign to the second y-axis
))

# fig.add_trace(go.Scatter(
#     x=df.time,
#     y=df.vel_x,
#     name="Vel x (m/s)",
#     yaxis="y2"  # Assign to the second y-axis
# ))

# fig.add_trace(go.Scatter(
#     x=df.time,
#     y=df.vel_y,
#     name="Vel y (m/s)",
#     yaxis="y2"  # Assign to the second y-axis
# ))

# fig.add_trace(go.Scatter(
#     x=df.time,
#     y=df.vel_z,
#     name="Vel z (m/s)",
#     yaxis="y2"  # Assign to the second y-axis
# ))

# fig.add_trace(go.Scatter(
#     x=df.time,
#     y=df.speed,
#     name="Speed",
#     yaxis="y2"  # Assign to the second y-axis
# ))


# add second y axis for antenna gain
# fig.add_trace(go.Scatter(x=df["time"], y=(tx_ant_gain+rx_ant_gain), name="Total Antenna Gain", yaxis="y2"))

fig.update_layout(
    yaxis2=dict(
        title="Velocity (m/s)",
        overlaying="y",  # Overlay the second y-axis on the first one
        side="right"     # Position the second y-axis on the right
    ))

pio.write_image(fig, RESULT_DIR+"PathLoss_vs_time.png",scale=6, width=1500, height=800)
fig.show()

In [ ]:
os.system(f"dolphin  {RESULT_DIR} &")

In [ ]:
# Plotting vs distance instead of time 
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["dist"], y=df["avg_pl"]-optimal_offset[2], mode="markers", name="Path Loss (dB)", marker_color=opac_colors[0]))
fig.add_trace(go.Scatter(x=df["dist"], y=df["avg_pl_ewma"]-optimal_offset[2], name=f"Path Loss (dB) Moving Average α={alpha}", marker_color=fl_colors[0]))
fig.add_trace(go.Scatter(x=df["dist"], y=pl_free, name="Free Space Path Loss", marker_color=opac_colors[12]))
fig.add_trace(go.Scatter(x=df["dist"], y=pl_2ray, name="2-Ray Path Loss", marker_color=opac_colors[36]))
fig.add_trace(go.Scatter(x=df["dist"], y=pl_free_log_normal, name="Free Space Path Loss (Log Normal)", marker_color=fl_colors[12]))
fig.add_trace(go.Scatter(x=df["dist"], y=pl_2ray_log_normal, name="2-Ray Path Loss (Log Normal)", marker_color=fl_colors[36]))
fig.add_trace(go.Scatter(x=df["dist"], y=pl_RMA_LoS, name="3GPP RMa LoS Path Loss", marker_color=fl_colors[46]))

fig.update_layout(
    autosize=False,
    width=1920,
    height=1080,
    title_text='Path Loss vs Distance',
    title_x=0.5,
    font=dict(family="courier new, monospace", size=18),
    xaxis_title="Distance (m)",
    yaxis_title="Path Loss (dB)",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

pio.write_image(fig, RESULT_DIR+"PathLoss_vs_Distance.png",scale=6, width=1500, height=800)

fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["time"], y=df["aod_phi"], name="AOD Elevation"))
fig.add_trace(go.Scatter(x=df["time"], y=df["aoa_phi"], name="AOA Elevation"))

fig.add_trace(go.Scatter(x=df["time"], y=df["aod_theta"], name="AOD Azimuth"))
fig.add_trace(go.Scatter(x=df["time"], y=df["aoa_theta"], name="AOA Azimuth"))

fig.update_layout(
    autosize=False,
    width=1920,
    height=1080,
    title_text='AOD/AOA vs Time',
    title_x=0.5,
    font=dict(family="Courier New, monospace", size=18),
    xaxis_title="Time (s)",
    yaxis_title="Angle (radians)",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()
pio.write_image(fig, RESULT_DIR+"AOD_AOA.png",scale=6, width=1920, height=1080)

In [ ]:
def calculate_endpoint(lat, lon, azimuth, distance=0.1):
    delta_lat = distance * np.cos(azimuth)
    delta_lon = distance * np.sin(azimuth)
    
    return lat + delta_lat, lon + delta_lon


In [ ]:
df["end_points"] = df.apply(lambda x: calculate_endpoint(x["lat"], x["lon"], x["aod_theta"],1e-4), axis=1)
df["end_lon"] = df["end_points"].apply(lambda x: x[1])
df["end_lat"] = df["end_points"].apply(lambda x: x[0])

In [ ]:
fig, ax = plt.subplots()

for i, _ in df.iterrows():
    if i  % 8 == 0:
        ax.arrow(df.lon[i], df.lat[i], df.end_lon[i] - df.lon[i], df.end_lat[i] - df.lat[i], width=1e-7, head_width=2e-5, head_length=3e-5, fc='red', ec='red')

ax.annotate('Source', xy=(LW1[1], LW1[0]), xytext=(LW1[1]+3e-5, LW1[0]+3e-5))

ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("AOD Azimuth")


ax.plot(LW1[1], LW1[0], marker='o', markersize=10, color='red')

fig.set_size_inches(20, 11)


fig.show()

fig.savefig(RESULT_DIR+"AOD_Azimuth.png")


In [ ]:


px.set_mapbox_access_token(
    MAPBOX_API_KEY
)

fig = px.scatter_mapbox(
    lat=df.lat,
    lon=df.lon,
    color=df.avgPower,
    labels={"avgPower": "Power(dBm)"},
    title=f"Power vs Location for Center Freq. = {
        freq} MHz and {waveType} waveform",
    mapbox_style="satellite",
    color_continuous_scale=px.colors.sequential.thermal,
    size_max=11,
    size=df["lat"],
    zoom=16.2,
    width=1400,
    height=1000
)

for i, _ in df.iterrows():
    if i % 10 == 0:
        fig.add_trace(
            go.Scattermapbox(
                mode="markers+lines",
                lon=[df.lon[i], df.end_lon[i]],
                lat=[df.lat[i], df.end_lat[i]],
                marker=dict(
                    symbol="arrow-up",
                    size=20
                ),
                line={"width": 3},
            )
        )
        
        
fig.add_trace(go.Scattermapbox(
    mode="markers",
    lon=[LW1[1]],
    lat=[LW1[0]],
    marker=dict(
        size=20,
        color="red"
    ),
    name="Source"
))

fig.add_annotation(
    go.layout.Annotation(
        text="Source",
        x=LW1[1]+1e-5,
        y=LW1[0]+1e-5,
        showarrow=False
    )
)

fig.update_layout(showlegend=False)

fig.show()

In [ ]:
plt.plot(df["time"], np.rad2deg(df["pitch"]))
plt.plot(df["time"], np.rad2deg(df["roll"]))
plt.plot(df["time"], np.rad2deg(df["yaw"]))


plt.legend(["Pitch", "Roll", "Yaw"])
plt.xlabel("Time (s)")
plt.ylabel("Angle (degrees)")

plt.savefig(RESULT_DIR+"Angles.png")

In [ ]:
lambda_ = 0.3  # Wavelength in meters (corresponds to 3.5 GHz frequency)
ht100 = 100  # Transmitter height for the first scenario in meters
ht30 = 30    # Transmitter height for the second scenario in meters
ht2 = 2      # Transmitter height for the third scenario in meters
hr = 2       # Receiver height in meters

# Initialize lists for storing results
distances = []
p100 = []
p30 = []
p2 = []
pfsl = []

# Calculate path loss for different distances
for i in range(30, 451):
    d = 10 ** (i / 1000)
    distances.append(d)
    fspower = (lambda_ / (4 * np.pi * d)) ** 2
    power100 = fspower * 4 * (np.sin(2 * np.pi * hr * ht100 / (lambda_ * d))) ** 2
    power30 = fspower * 4 * (np.sin(2 * np.pi * hr * ht30 / (lambda_ * d))) ** 2
    power2 = fspower * 4 * (np.sin(2 * np.pi * hr * ht2 / (lambda_ * d))) ** 2
    p100.append(20 * np.log10(power100))
    p30.append(20 * np.log10(power30))
    p2.append(20 * np.log10(power2))
    pfsl.append(20 * np.log10(fspower))

# Plotting
plt.figure(figsize=(10, 6))
plt.semilogx(distances, p100, 'g-', label='ht=100m')
plt.semilogx(distances, p30, 'b-', label='ht=30m')
plt.semilogx(distances, p2, 'r-', label='ht=2m')
plt.semilogx(distances, pfsl, 'y-', label='Free Space')
plt.xlabel('Distance (m)')
plt.ylabel('Path Loss (dB)')
plt.title('Two-Ray Ground Reflection Model Path Loss')
plt.legend()
plt.grid(True)
plt.show()

In [45]:
df['section'] = 'flight'  # Default to flight

df.loc[df['vel_z'] < -TO_THRESHOLD-0.2, 'section'] = 'takeoff'
df.loc[df['vel_z'] > TO_THRESHOLD+0.2, 'section'] = 'landing'

takeoff_start = df[df['section'] == 'takeoff'].index[0]
df.loc[:takeoff_start, 'section'] = 'takeoff'

# Label the section after landing as 'landing'
landing_end = df[df['section'] == 'landing'].index[0]
df.loc[landing_end:, 'section'] = 'landing'


In [ ]:
RESULT_DIR = prcsd["resultDir"][0]

color = ["#1f77b4", "#ff7f0e", "#2ca02c"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.time, y=df.avg_pl-optimal_offset[2]+3, mode="markers", name="Path Loss Measurement"))
fig.add_trace(go.Scatter(x=df.time, y=df.avg_pl_rolling-optimal_offset[2]+3, name=f"EWMA α={alpha} Path Loss Measurement", mode="lines"))

# fig.add_trace(go.Scatter(x=df.time+2, y=pl_free,
#                     mode='lines',
#                     name='FSPL',
#                     yaxis="y1"))

# fig.add_trace(go.Scatter(x=df.time+2, y=pl_2ray,
#                     mode='lines',
#                     name='2Ray',yaxis="y1"))

fig.add_trace(go.Scatter(x=df.time, y=pl_free_log_normal,
                    mode='lines',
                    name='FSPL w/ Log Normal',yaxis="y1"))
fig.add_trace(go.Scatter(x=df.time, y=pl_2ray_log_normal,
                    mode='lines',
                    name='2Ray w/ Log Normal',yaxis="y1"))

fig.add_trace(go.Scatter(x=df.time, y=pl_RMA_LoS,
                    mode='lines',
                    name='3GPP RMa LoS',yaxis="y1"))


fig.add_trace(go.Scatter(
    x=df.time,
    y=df.alt,
    name="Altitude",
    yaxis="y2"  # Assign to the second y-axis
))

# fig.add_trace(go.Scatter(
#     x=df.time,
#     y=df.aoa_theta,
#     name="AOA Theta",
#     yaxis="y2"  # Assign to the second y-axis
# ))

colors = {'takeoff': 'rgba(135, 206, 235, 0.5)',
        'flight': 'rgba(144, 238, 144, 0.5)',
        'landing': 'rgba(135, 206, 235, 0.5)'}

pl_indices = ["avg_pl"]

# Add shapes for the sections
for section, color in colors.items():
    section_indices = df[df['section'] == section].index
    if not section_indices.empty:
        start, end = section_indices[0], section_indices[-1]
        fig.add_shape(type='rect',
                    x0=df.loc[start, 'time'], x1=df.loc[end, 'time'], y0=df[pl_indices].min(axis=1).min()-optimal_offset[2]-13, y1=df[pl_indices].max(axis=1).max()-optimal_offset[2]+6,
                    fillcolor=color, opacity=0.5, line_width=0)
        fig.add_annotation(x=(df.loc[start, 'time'] + df.loc[end, 'time']) / 2,
                    y=df['avg_pl'].max()+5.2,
                    text=section.capitalize(),
                    showarrow=False,
                    yshift=400,
                    font=dict(color='gray', size=24))

fig.update_layout(
    autosize=False,
    width=2000,
    height=1000,
    # title_text=f"Path Loss vs Time for Center Freq. = {freq} MHz and {waveType} waveform",
    font=dict(family="Courier New, monospace", size=18),
    xaxis_title="Time (s)",
    yaxis_title="Path Loss (dB)"
)
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    yaxis=dict(
    ),
    yaxis2=dict(
        title="Altitude (m)",
        overlaying="y",  # Overlay the second y-axis on the first one
        side="right"     # Position the second y-axis on the right
    ))


fig.show()
pio.write_image(fig, RESULT_DIR+"PathLossvsTime.png", width=2500, height=1000)
pio.write_image(fig, RESULT_DIR+"PathLossvsTime.pdf", width=2500, height=1000)

In [ ]:
os.system(f"dolphin  {RESULT_DIR} &")

In [ ]:
from scipy import stats
from fitter import Fitter

f = Fitter((df["avg_pl"]-optimal_offset[2]).tolist(), distributions=['gamma', 'rayleigh', 'lognorm', 'norm', 'expon', 'beta'])
f.fit()

fig, ax0 = plt.subplots(ncols=1, nrows=1, figsize=(12, 8))

(values, bins, _) = ax0.hist(df["avg_pl"]-optimal_offset[2], bins=25, density=True, color="blue", alpha=0.7, label="Path Loss")
x = np.linspace(bins.min(), bins.max(), len(f.fitted_pdf["lognorm"]))
ax0.plot(x, f.fitted_pdf["lognorm"], color="black", label="Lognormal Fit")
ax0.hist(pl_free, bins=25, color="red", density=True, alpha=0.7, label="Free Space Path Loss")
ax0.hist(pl_2ray, bins=25, color="green", density=True, alpha=0.7, label="2-Ray Path Loss")

ax0.legend()
ax0.set_title("Path Loss PDF")
ax0.xaxis.set_label_text("Path Loss (dB)")
ax0.yaxis.set_label_text("Density")

In [ ]:
fig, ax0 = plt.subplots(ncols=1, nrows=1, figsize=(12, 8))

H,X1 = np.histogram(df["avg_pl"]-optimal_offset[2], bins = 30, density= True )
dx = X1[1] - X1[0]
F1 = np.cumsum(H)*dx

ax0.plot(X1[1:], F1, color="purple", label="Path Loss")

params = f.fitted_param['lognorm']

s = params[0]  # sigma (shape)
loc = params[1]  # location
scale = params[2]  # scale

dist = stats.lognorm(s=s, loc=loc, scale=scale)

ax0.plot(x, dist.cdf(x), color="red", label="Lognormal Fit")

H_fs, X1_fs = np.histogram(pl_free, bins=30, density=True)
dx_fs = X1_fs[1] - X1_fs[0]
F1_fs = np.cumsum(H_fs) * dx_fs

F1_fs = np.concatenate((F1_fs,np.array([1.0])))
X1_fs = np.concatenate((X1_fs, X1[-1:]))

ax0.plot(X1_fs[1:], F1_fs, color="blue", label="Free Space Path Loss")

H_2ray, X1_2ray = np.histogram(pl_2ray, bins=30, density=True)
dx_2ray = X1_2ray[1] - X1_2ray[0]
F1_2ray = np.cumsum(H_2ray) * dx_2ray

F1_2ray = np.concatenate((F1_2ray, np.array([1.0])))
X1_2ray = np.concatenate((X1_2ray, X1[-1:]))

ax0.plot(X1_2ray[1:], F1_2ray, color="green", label="2-Ray Path Loss") 

ax0.legend()
ax0.set_title("Path Loss CDF")
ax0.xaxis.set_label_text("Path Loss (dB)")
ax0.yaxis.set_label_text("Probability")

ax0.annotate(f"Lognormal Fit: s={s:.2f}, loc={loc:.2f}, scale={scale:.2f}", xy=(0.59, 0.15), xycoords='axes fraction', fontsize=12,color="red", ha='center', va='center')
ax0.annotate(f"Free Space Path Loss: μ={pl_free.mean():.2f}, σ={pl_free.std():.2f}", xy=(0.59, 0.1), xycoords='axes fraction', fontsize=12, color="blue", ha='center', va='center')
ax0.annotate(f"2-Ray Path Loss: μ={pl_2ray.mean():.2f}, σ={pl_2ray.std():.2f}", xy=(0.59, 0.05), xycoords='axes fraction', fontsize=12, color="green", ha='center', va='center')


fig.show()

In [ ]:
f.fitted_param['lognorm']

In [ ]:
px.set_mapbox_access_token(MAPBOX_API_KEY)

fig = px.scatter_mapbox(df, lat="lat", lon="lon", color="avg_pl", labels={"path_loss": "Path Loss(dB)"}, title="Path Loss Between CC1 and PN",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=10, zoom=17.5,  width=1300, height=1000)
fig.show()
pio.write_image(fig, RESULT_DIR+"PL_GPS.png", scale=6, width=2000, height=1000)

# CIR 

In [ ]:
fig = plt.figure(figsize=(30, 30))
ax = fig.add_subplot(projection='3d')

st = 50
end = 100
st_i = 70000
m = 0

colors = ['r', 'g', 'b', 'y']
for row in df.iloc[st:end].iterrows():
    zero_ind = row[1]["peaks"][0]

    if zero_ind < st_i:
        st_i = zero_ind
    else:
        zero_ind = st_i
        
    cr =  row[1]["corr"][zero_ind-2350:zero_ind+50]
    cr = np.array(cr)

    # cr[cr < -20] = -20
    # cr += 20
    # cr /= np.argmax(cr)
    
    if(len(cr) > m):
        m = len(cr)
        
    ts = np.linspace(0,(1/sample_rate)*m*1e6,m)

    ax.plot(ts, cr, zs=row[1]["time"], zdir='y', alpha=1)
    

ax.set_xlabel('Time (μs)')
ax.set_ylabel('Samples')
ax.set_zlabel('CIR')

plt.savefig(RESULT_DIR+f"CIR({st}-{end}s).pdf", dpi=300, format='pdf', bbox_inches='tight')
plt.show()


In [ ]:
cir = df["corr"].copy()

# zero_ind = df["peaks"][100][0]
_cir_off = df["start_point"].min()

max_delay = []
x_line = []
y_line = []
for i in range(len(cir)):
    # _st = df["orig_peaks"][i][1] - _cir_off - 100
    cr =  df["corr"][i].astype(float)
    cr = np.array(10*np.log10(np.abs(cr)+1e-8))
    # cr = np.array(np.abs(cr))
    # cr = np.roll(cr, _st)
    x_line.append(df["dist"][i]/250)
    y_line.append(df["time"][i])
    cr = cr[560:700]
    threshold = 0.3 * np.max(np.abs(cr))

    # Identify significant paths based on the threshold
    significant_indices = np.where(np.abs(cr) > threshold)[0]
    time_indices = np.linspace(0, len(cr), len(cr)) * (1/cfg.USRP_CONF.SAMPLE_RATE) * 1e6
    # Calculate max excess delay
    if significant_indices.size > 0:
        max_excess_delay = time_indices[significant_indices[-1]] - time_indices[significant_indices[0]]
    else:
        max_excess_delay = 0
    
    max_delay.append(max_excess_delay)
    # cr -= cr.mean()
    cir[i] = cr

z_line = np.zeros_like(x_line) + 23

print(f"Average Max Delay: {np.mean(max_delay)} (μs)")
x_ax = np.linspace(0, len(cr), len(cr)) * (1/cfg.USRP_CONF.SAMPLE_RATE) * 1e6

line = go.Scatter3d(
    x=x_line,
    y=y_line,
    z=z_line,
    mode='lines',
    line=dict(color='black', width=8)
)

fig = go.Figure(data=[go.Surface(z=cir,y=df["time"],x=x_ax, colorscale='Turbo', 
                      colorbar=dict(title=dict(text='Magnitude<br>(dBm)', font=dict(size=16)))), line])
                      
fig.update_layout(autosize=False,
                  width=1000, height=1000)

# change font size of the axis titles 
fig.update_layout(scene = dict(
                    xaxis_title='Sample Time (μs)',
                    yaxis_title='Experiment Time (s)',
                    zaxis_title='Magnitude (dBm)'),
                    font=dict(size=16),
                    margin=dict(r=10, b=10, l=10, t=10),
)



fig.update_layout(scene_camera=dict(
    up=dict(x=0, y=0, z=2),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0.6, y=1.6, z=1.35)
))


# Create a subplot with 1 row and 2 columns

# fig.update_layout(
#     xaxis2=dict(
#         overlaying='x',
#         side='top',
#         title='Secondary X Axis'
#     ),
#     yaxis2=dict(
#         overlaying='y',
#         side='right',
#         title='Secondary Y Axis'
#     )
# )
fig.add_annotation(
    dict(
        x=0.25,
        y=0.94,
        textangle=-7.5,
        text="GPS Distance over time (m)",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)

fig.add_annotation(
    dict(
        x=0.64,
        y=0.93,
        textangle=-6,
        text="0",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)

fig.add_annotation(
    dict(
        x=0.52,
        y=0.92,
        textangle=-6,
        text="130",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)

fig.add_annotation(
    dict(
        x=0.42,
        y=0.91,
        textangle=-6,
        text="260",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)

fig.add_annotation(
    dict(
        x=0.31,
        y=0.895,
        textangle=-6,
        text="390",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)

fig.add_annotation(
    dict(
        x=0.19,
        y=0.883,
        textangle=-6,
        text="520",
        showarrow=False,
        xanchor="left",
        xref="paper",
        yref="paper",
        font=dict(size=16)
    )
)


fig.show()

fig.write_html( RESULT_DIR+"CIRvsTime3D.html")
# pio.write_image(fig, RESULT_DIR+"CIRvsTime3D.pdf",scale=4, width=500, height=500)

In [ ]:
plt.plot(df["time"], df["dist"])

In [ ]:
df["peaks"]

In [ ]:
df["orig_peaks"]

In [ ]:
df = prcsd["meas"][0]
cir = df["corr"].copy() 
zero_ind = df["start"][50].copy()


for i in range(0,len(cir)//100):
    cr =  df["corr"][i][zero_ind-2:zero_ind+200]
    cr = np.array(cr)
    # cr[cr < np.mean(cr)] = 0
    # cr = signal.resample(cr, len(cr)*2)
    # if cr.mean() > m :
    #     cr[cr < 1] = 0
    # cr[cr < -20.5] = 0
    # ts = np.linspace(0,(1/sample_rate)*len(cr)*1e6,len(cr))
    cir[i] = cr
#,colorscale='Jet'

fig = go.Figure(data=[go.Surface(z=cir, y=df["time"], colorscale='Turbo')])
fig.update_layout(autosize=False,
                  font=dict(family="Courier New, monospace", size=15),
                  width=1000, height=1000)

fig.update_layout(scene = dict(
                    xaxis_title='Taps',
                    yaxis_title='Time (s)',
                    zaxis_title='CIR'),
                    margin=dict(r=10, b=10, l=10, t=10) )

fig.update_layout(scene_camera=dict(
    up=dict(x=0, y=0, z=2),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0.6, y=1.6, z=1.35)
))

fig.show()
print(RESULT_DIR)
# pio.write_image(fig, RESULT_DIR+"CIRvsTime3D.pdf", width=600, height=600)
fig.write_html( RESULT_DIR+"CIRvsTime3D.html")

In [ ]:
RESULT_DIR

In [ ]:
df = prcsd["meas"][0]
cir = df["_xcorr"].copy() 
st_i = 50000

for i in range(len(cir)):
    zero_ind = df["start"][i].copy()
    # if zero_ind < st_i:
    #     st_i = zero_ind
    # else:
    #     zero_ind = st_i
    cr =  df["_xcorr"][i][zero_ind+10:zero_ind+200]

    cr = np.array(cr)
    # if cr.mean() > m_3 :
    #  cr[cr < m_3] = m_3
    cir[i] = np.log10(abs(cr))
    # ts = np.linspace(0,(1/sample_rate)*len(cr)*1e6,len(cr))
    # cir[i] = cr


fig = go.Figure(data=[go.Surface(z=cir,colorscale='Jet')])
fig.update_layout(autosize=False,
                  width=1000, height=1000)
fig.update_layout(scene = dict(
                    xaxis_title='Taps',
                    yaxis_title='Sample Number',
                    zaxis_title='CIR (dB)'),
                    margin=dict(r=10, b=10, l=10, t=10) )

fig.update_layout(scene_camera=dict(
    up=dict(x=0, y=0, z=2),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0.6, y=1.6, z=1.35)
))


fig.show()

In [ ]:
df = prcsd["meas"][0]
cir = df["cir"].copy() 
st_i = 50000

old = np.seterr(invalid='ignore')

for i in range(len(cir)):
    zero_ind = df["start"][i].copy()
    # if zero_ind < st_i:
    #     st_i = zero_ind
    # else:
    #     zero_ind = st_i
    cr =  df["cir"][i][2400:3500]

    cr = np.array(cr)

    cr = np.nan_to_num(cr, nan=-0.0001)

    if cr.mean() > m_2:
        cr[cr < 1] = m_2_min
        
    cir[i] = -np.log10(abs(cr))
    # cr[cr < -20.5] = 0
    # ts = np.linspace(0,(1/sample_rate)*len(cr)*1e6,len(cr))
    


fig = go.Figure(data=[go.Surface(z=cir)])
fig.update_layout(autosize=False,
                  width=1000, height=1000)
fig.update_layout(scene = dict(
                    xaxis_title='Taps',
                    yaxis_title='Sample Number',
                    zaxis_title='CIR'),
                    margin=dict(r=10, b=10, l=10, t=10) )

fig.update_layout(scene_camera=dict(
    up=dict(x=0, y=0, z=2),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0.6, y=1.6, z=1.35)
))


fig.show()

In [ ]:
pio.write_image(fig, RESULT_DIR+"CIRvsTime3D_2.png",scale=4, width=600, height=600)

In [ ]:
df = prcsd["meas"][0]
cir = df["cir"].copy() 
st_i = 50000
a = []
for i in range(len(cir)):
    zero_ind = df["start"][i].copy()
    # if zero_ind < st_i:
    #     st_i = zero_ind
    # else:
    #     zero_ind = st_i
    cr =  df["cir"][i][2000:2000]

    cr = np.array(cr)
    
    cr[cr < -20.5] = 0
    # ts = np.linspace(0,(1/sample_rate)*len(cr)*1e6,len(cr))
    cir[i] = cr
    a.append(cr.mean())

m_2 = np.mean(a)
m_2_min = min(a)

In [ ]:
df = prcsd["meas"][0]
cir = df["corr"].copy() 
st_i = 50000
a = []
for i in range(len(cir)):
    zero_ind = df["start"][i].copy()
    # if zero_ind < st_i:
    #     st_i = zero_ind
    # else:
    #     zero_ind = st_i
    cr =  df["corr"][i][2000:4000]

    cr = np.array(cr)
    
    cr[cr < -20.5] = 0
    # ts = np.linspace(0,(1/sample_rate)*len(cr)*1e6,len(cr))
    cir[i] = cr
    a.append(cr.mean())
m = np.mean(a)

In [ ]:
df = prcsd["meas"][0]
cir = df["_xcorr"].copy() 
st_i = 50000
a = []
for i in range(len(cir)):
    zero_ind = df["start"][i].copy()
    # if zero_ind < st_i:
    #     st_i = zero_ind
    # else:
    #     zero_ind = st_i
    cr =  df["_xcorr"][i][2000:4000]

    cr = np.array(cr)
    
    cr[cr < -20.5] = 0
    # ts = np.linspace(0,(1/sample_rate)*len(cr)*1e6,len(cr))
    cir[i] = cr
    a.append(cr.mean())
m_3 = np.mean(a)
m_3_min = min(a)

# TEST AREA

In [ ]:

def test_plot_cir(fpath): 
    # r = np.load("../sounder/exp/out/2022-11-08/received_120.npz")
    r = np.load(fpath)
    rcv = r["rcv"][0]
    x = r["ref"]# wave key for new version

    xcorr = signal.correlate(x, rcv, mode="full", method="fft")
    xcorr = np.flip(xcorr)
    lags = 1 / signal.correlation_lags(len(x), len(rcv))
    xcorr = np.abs(xcorr)
    xcorr /= np.max(xcorr)

    corrI = signal.correlate(rcv.real, x.real, mode="full", method="fft") 
    corrI = corrI/len(x)

    corrQ = signal.correlate(rcv.imag, x.imag, mode="full", method="fft")
    corrQ = corrQ/len(x)

    corrI = corrI ** 2
    corrQ = corrQ ** 2
    corrIQ = np.array([x + y for x, y in zip(corrI, corrQ)])
    cir = np.sqrt(corrIQ)

    # cir /= np.max(cir)

    # dist = geodesic(cc1, (r["lat"], r["lon"])).meters
    zero_index_cir = np.argmax(xcorr)
    dist_calc = (zero_index_cir - 2612) * 1/sample_rate * 3e8
    print(dist_calc, zero_index_cir)
    power = 10*np.log10(np.abs(np.sqrt(np.mean(rcv[:len(x)*2] ** 2)))) 
    print(f"Power {power} dBFS")

    plt.rc('axes', titlesize=19)
    plt.rc('axes', labelsize=19)
    plt.rc('xtick', labelsize=19) 
    plt.rc('ytick', labelsize=19)
    fig, (cir_plot) = plt.subplots(1, 1, figsize=(12, 5))
    
    m_cir = xcorr[zero_index_cir-3:zero_index_cir+650]
    cir_plot.plot(np.linspace(0,(1/sample_rate)*len(m_cir)*1e6,len(m_cir)),(m_cir))
    cir_plot.set_title('CIR')
    cir_plot.set_xlabel("Time (μs)")
    cir_plot.set_xlabel("Time (μs)")
    cir_plot.margins(0, 0.1)

    peaks_y = np.sort(m_cir)[-3:]
    peaks_y = np.delete(peaks_y, 1)
    sorter = np.argsort(m_cir)
    peaks_x = sorter[np.searchsorted(m_cir, peaks_y, sorter=sorter)] * 1/sample_rate* 1e6 + 2.5e-3

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=np.linspace(0, (1 / sample_rate) * len(m_cir) * 1e6, len(m_cir)),
            y=m_cir,
            mode="lines",
            name="lines",
        )
    )
    fig.update_traces(line=dict(width=4))
    fig.add_trace(go.Scatter(x=peaks_x, y=peaks_y, mode="markers", name="markers"))
    fig.update_traces(marker=dict(size=16), selector=dict(mode="markers"))

    fig.add_annotation(
        x=peaks_x[-1],
        y=peaks_y[-1] + 1.2e-2,
        text="LOS Path",
        showarrow=True,
        arrowhead=2,
        arrowwidth=4,
        arrowcolor="#636363",
        font=dict(family="Courier New, monospace", size=54, color="#ffffff"),
        ax=220,
        ay=-70,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#CC0000",
        opacity=0.8,
    )

    fig.add_annotation(
        x=peaks_x[-2],
        y=peaks_y[-2] + 1.2e-2,
        text="2nd Path",
        showarrow=True,
        arrowhead=2,
        arrowwidth=4,
        arrowcolor="#636363",
        font=dict(family="Courier New, monospace", size=54, color="#ffffff"),
        ax=220,
        ay=-40,
        align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#CC0000",
        opacity=0.8,
    )

    # fig.add_annotation(
    #     x=peaks_x[-3],
    #     y=peaks_y[-3] + 1.7e-2,
    #     text="3rd Path",
    #     showarrow=True,
    #     arrowhead=2,
    #     arrowwidth=4,
    #     arrowcolor="#636363",
    #     font=dict(family="Courier New, monospace", size=54, color="#ffffff"),
    #     ax=220,
    #     ay=-50,
    #     align="center",
    #     bordercolor="#c7c7c7",
    #     borderwidth=2,
    #     borderpad=4,
    #     bgcolor="#CC0000",
    #     opacity=0.8,
    # )

    fig.update_layout(
        autosize=False,
        width=1600,
        height=800,
        margin=dict(l=30, r=30, b=30, t=50, pad=5),
        title_x=0.5,
        font=dict(family="Courier New, monospace", size=40),
        xaxis_title="Time (μs)",
        yaxis_title="Normalized CIR",
        plot_bgcolor = "white"
    )

    fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=3, linecolor='black')
    fig.update_layout(showlegend=False)

    fig.update_layout(xaxis_range=[0, 7])
    fig.update_xaxes(color="black")

    fig.show()
    RESULT_DIR = "../results/" + fpath.split("/")[2] + "/"
    pio.write_image(
        fig, RESULT_DIR + "Normalized_CIR_6_6_6.png", scale=6, width=1800, height=800
    )

    # plt.plot(rcv[:500])

    # plt.savefig(RESULT_DIR+f"CIR_11_8_22_122.png", dpi=800, format='png', bbox_inches='tight')

    plt.show()

In [ ]:
test_plot_cir("../measurements/2023-09-29_18_26/received_60.01.npz")

In [ ]:
z_ind = np.argmax(xcorr)
sig_of_int = rcv[z_ind-135:z_ind+20]
# power = 20 * np.log10(np.var(sig_of_int))
power = 10 * np.log10(np.abs(np.sqrt(np.mean(sig_of_int ** 2)))) 
10*np.log10(np.abs(np.sqrt(np.mean(x ** 2)))/np.abs(np.sqrt(np.mean(sig_of_int ** 2))))

In [ ]:
peaks_y = np.sort(m_cir)[-4:]
peaks_y = np.delete(peaks_y, 2)
sorter = np.argsort(m_cir)
peaks_x = sorter[np.searchsorted(m_cir, peaks_y, sorter=sorter)] * 1/sample_rate* 1e6 + 2.5e-3
# peaks_y -= 2e-2

## Special Plots

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.linspace(0, (1 / sample_rate) * len(m_cir) * 1e6, len(m_cir)),
        y=m_cir,
        mode="lines",
        name="lines",
    )
)
fig.update_traces(line=dict(width=4))
fig.add_trace(go.Scatter(x=peaks_x, y=peaks_y, mode="markers", name="markers"))
fig.update_traces(marker=dict(size=16), selector=dict(mode="markers"))

fig.add_annotation(
    x=peaks_x[-1],
    y=peaks_y[-1] + 1.5e-2,
    text="Line of Sight Path",
    showarrow=True,
    arrowhead=2,
    arrowwidth=4,
    arrowcolor="#636363",
    font=dict(family="Courier New, monospace", size=54, color="#ffffff"),
    ax=320,
    ay=-70,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#CC0000",
    opacity=0.8,
)

fig.add_annotation(
    x=peaks_x[-2],
    y=peaks_y[-2] + 1.7e-2,
    text="2nd Path",
    showarrow=True,
    arrowhead=2,
    arrowwidth=4,
    arrowcolor="#636363",
    font=dict(family="Courier New, monospace", size=54, color="#ffffff"),
    ax=220,
    ay=-60,
    align="center",
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#CC0000",
    opacity=0.8,
)

# fig.add_annotation(
#     x=peaks_x[-3],
#     y=peaks_y[-3] + 1.7e-2,
#     text="3rd Path",
#     showarrow=True,
#     arrowhead=2,
#     arrowwidth=4,
#     arrowcolor="#636363",
#     font=dict(family="Courier New, monospace", size=54, color="#ffffff"),
#     ax=220,
#     ay=-50,
#     align="center",
#     bordercolor="#c7c7c7",
#     borderwidth=2,
#     borderpad=4,
#     bgcolor="#CC0000",
#     opacity=0.8,
# )

fig.update_layout(
    autosize=False,
    width=1600,
    height=800,
    margin=dict(l=30, r=30, b=30, t=50, pad=5),
    title_x=0.5,
    font=dict(family="Courier New, monospace", size=40),
    xaxis_title="Time (μs)",
    yaxis_title="Coefficient",
    plot_bgcolor = "white"
)

fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
fig.update_yaxes(showline=True, linewidth=3, linecolor='black')
fig.update_layout(showlegend=False)

fig.update_layout(xaxis_range=[0, 3.5])
fig.update_xaxes(color="black")

fig.show()
pio.write_image(
    fig, RESULT_DIR + "Normalized_CIR_6_6_6.png", scale=6, width=1800, height=800
)


In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=df.time, y=df.power,
                    mode='lines',
                    name='Power'))


fig.update_layout(
    autosize=False,
    width=1500,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ), 
    title_text='Power',
    xaxis_title="Time (s)",
    yaxis_title="Power (dB)"
)

fig.show()

In [ ]:
10*np.log10(np.abs(np.sqrt(np.mean(sig ** 2)))) 

In [ ]:
1/sample_rate*10*3e8

In [ ]:
fig = px.scatter_mapbox(df, lat="lat", lon="lon", color="power", labels={"power": "Power(dB)"}, title="Power Distribution Between CC1 and PN",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=10, zoom=17.4,  width=2000, height=1000)
fig.show()
pio.write_image(fig, RESULT_DIR+"Power_GPS.png", scale=6, width=2000, height=1000)

In [ ]:
len(xcorr)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

colors = ['r', 'g', 'b', 'y']
yticks = [3, 2, 1, 0] 
for c, k in zip(colors, yticks):
    # Generate the random data for the y=k 'layer'.
    xs = np.arange(20)
    ys = np.random.rand(20)

    # You can provide either a single color or an array with the same length as
    # xs and ys. To demonstrate this, we color the first bar of each set cyan.
    cs = [c] * len(xs)
    cs[0] = 'c'

    # Plot the bar graph given by xs and ys on the plane y=k with 80% opacity.
    ax.bar(xs, ys, zs=k, zdir='y', color=cs, alpha=0.8)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# On the y axis let's only label the discrete values that we have data for.
ax.set_yticks(yticks)

plt.show()